In [30]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from sklearn.model_selection import train_test_split
from tqdm import tqdm

np.random.seed(2022)

In [44]:
def eliq(T):
    a_liq = np.float32(np.array([-0.976195544e-15,-0.952447341e-13,\
                                 0.640689451e-10,\
                      0.206739458e-7,0.302950461e-5,0.264847430e-3,\
                      0.142986287e-1,0.443987641,6.11239921]));
    c_liq = np.float32(-80.0)
    T0 = np.float32(273.16)
    return np.float32(100.0)*np.polyval(a_liq,np.maximum(c_liq,T-T0))

def eice(T):
    a_ice = np.float32(np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,\
                      0.602588177e-7,0.615021634e-5,0.420895665e-3,\
                      0.188439774e-1,0.503160820,6.11147274]));
    c_ice = np.float32(np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07]))
    T0 = np.float32(273.16)
    return xr.where(T>c_ice[0],eliq(T),\
                   xr.where(T<=c_ice[1],np.float32(100.0)*(c_ice[3]+np.maximum(c_ice[2],T-T0)*\
                   (c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5])),\
                           np.float32(100.0)*np.polyval(a_ice,T-T0)))
def esat(T):
    T0 = np.float32(273.16)
    T00 = np.float32(253.16)
    omtmp = (T-T00)/(T0-T00)
    omega = np.maximum(np.float32(0.0),np.minimum(np.float32(1.0),omtmp))
    return xr.where(T>T0,eliq(T),xr.where(T<T00,eice(T),(omega*eliq(T)+(1-omega)*eice(T))))

def RH(T,qv,P0,PS,hyam,hybm):
    R = np.float32(287.0)
    Rv = np.float32(461.0)
    p = P0 * hyam + PS[:, None] * hybm # Total pressure (Pa)
    
    T = np.float32(T)
    qv = np.float32(qv)
    p = np.float32(p)
    
    return Rv*p*qv/(R*esat(T))

In [45]:
datasets = !ls
datasets = [x for x in datasets if "h1" in x]
spData = xr.open_mfdataset(datasets)

In [ ]:
nntbp = spData["NNTBP"]
nnqbp = spData["NNQBP"]
p0 = spData["P0"]
ps = spData["NNPS"]
hyam = spData["hyam"]
hybm = spData["hybm"]
relhum = spData["RELHUM"]
tphystnd = spData["TPHYSTND"]
phq = spData["PHQ"]
newhum = xr.zeros_like(spData["NNQBP"])
lats = spData["lat"]
lons = spData["lon"]
for i in tqdm(range(len(lats))):
    for j in range(len(lons)):
        latIndex = i
        lonIndex = j
        R = np.float32(287.0)
        Rv = np.float32(461.0)
        p = p0 * hyam + ps[:, latIndex, lonIndex].expand_dims({"lev":30}, axis = 1)*hybm
        T = nntbp[:, :, latIndex, lonIndex]
        qv = nnqbp[:, :, latIndex, lonIndex]
        newhum[:,:, latIndex, lonIndex] = Rv*p*qv/(R*esat(T))

In [52]:
latIndex = 8
lonIndex = 45
R = 287
Rv = 461
p = p0 * hyam + ps[:, latIndex, lonIndex].expand_dims({"lev":30}, axis = 1)*hybm
T = nntbp[:, :, latIndex, lonIndex]
qv = nnqbp[:, :, latIndex, lonIndex]
hmm = Rv*p*qv/(R*esat(T))
wat = hmm.values

In [53]:
wat

array([[1.91905080e-05, 1.25472810e-04, 8.95504162e-04, ...,
        9.61533012e-01, 9.49013535e-01, 9.61649122e-01],
       [1.91017488e-05, 1.24586185e-04, 8.97240213e-04, ...,
        9.62264978e-01, 9.46424741e-01, 9.58811529e-01],
       [1.89939755e-05, 1.23529960e-04, 8.96677226e-04, ...,
        9.63038529e-01, 9.45271102e-01, 9.55126566e-01],
       ...,
       [2.11757356e-05, 1.33481361e-04, 1.28208615e-03, ...,
        8.85127583e-01, 9.31234704e-01, 9.69864523e-01],
       [2.11639513e-05, 1.33360727e-04, 1.28696974e-03, ...,
        8.82691742e-01, 9.30808413e-01, 9.78882685e-01],
       [2.10799304e-05, 1.33185410e-04, 1.29137741e-03, ...,
        8.71685640e-01, 9.35304284e-01, 9.78382564e-01]])

In [56]:
T0 = np.float32(273.16)
c_ice = np.float32(np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07]))
np.maximum(c_ice[2],T-T0)

<xarray.DataArray 'NNTBP' (time: 96, lev: 30)>
dask.array<maximum, shape=(96, 30), dtype=float32, chunksize=(48, 30), chunktype=numpy.ndarray>
Coordinates:
    lat      float64 -65.58
    lon      float64 126.6
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * time     (time) object 0000-11-04 00:00:00 ... 0000-11-05 23:30:00